In [ ]:
import os
import pandas as pd

### Foursquare & Yelp venues

In [ ]:
#custom dict of category ids and labels
cat_ids = [['Arts and Entertainment', 10000, 'arts, All'],
           ['Business and Professional Services', 11000, 'business, All'],
           ['Community and Government', 12000, 'publicservicesgovt, All'],
           ['Dining and Drinking', 13000, 'restaurants, All'],
           ['Event', 14000,'yelpevents, All'],
           ['Health and Medicine', 15000,'health, All'],
           ['Landmarks and Outdoors', 16000,'landmarks, All'],
           ['Retail', 17000,'shopping, All'],
           ['Sports and Recreation', 18000,'active, All'],
           ['Travel and Transportation', 19000,'transport, All']
          ]

cat_ids = pd.DataFrame(cat_ids, columns=['Category','four_ID','yelp_ID'])

#### Foursquare

In [ ]:
headers_four = {
    "Accept": "application/json",
    "Authorization": os.environ["FOURSQUARE_API_KEY"]
}

In [ ]:
#make a function that returns venue names for all venue types and total venue amount
def get_venues_four(location, category):
    """ intakes: string of lat, long location / int of acceptable category, see cat_ids
        returns: Dataframe of all the subcategory types found in that category along with counts
    ex input: get_venues_four('45.6387,-122.6615', 13000)"""
    root_endpoint = "https://api.foursquare.com/v3/places/search"
    
    #get data relevent to input location
    response = re.get(root_endpoint, 
                      params={'ll': location, 'radius': 1000, 'categories': category, 'limit': 50}, 
                      headers = headers_four)
    result = response.json()
    
    #define empty objects
    category_amount = 0
    venues = {'id': [],
              'title': []
    }
    
    #cycle into data to get place type
    for place in result['results']:
        
        #go into every type a place is
        for p_type in place['categories']:
    
            category_amount +=1 
            venues['title'].append(p_type['name'])
            venues['id'].append(p_type['id'])

    #save venues into data frame and count values
    venues_df = pd.DataFrame(venues).value_counts()
    #return a database of the venue names and an integer of total tagged with that id
    return pd.DataFrame(venues_df, columns=['total'])

test_df = get_venues_four('45.6387,-122.6615', 13000)
test_df.head()

#### Yelp

In [ ]:
def get_venues_yelp(location, category):
    """ intakes: string of lat, long location / int of acceptable category, see cat_ids
        returns: Dataframe of all the subcategory types found in that category along with counts
    ex input: get_venues_four('45.6387,-122.6615', 13000)"""
    root_endpoint = "https://api.yelp.com/v3/businesses/search"
    
    #Test out retrieving API info
    response = re.get(root_endpoint, 
                  params = {'latitude': location[0], 
                            'longitude':location[1],
                            'categories':category,
                            'radius':1000},
                  headers = headers_yelp)
    
    result = response.json()
    
    #define empty objects
    venue_amount = 0
    venues = []
    
    for place in result['businesses']:
      
        #print(i['categories'])
        for venue_type in place['categories']:
        
            #print(venue_type['alias'])
            venues.append(venue_type['title'])
            venue_amount += 1
        

    #save venues into data frame and count values
    venues_df = pd.DataFrame(venues, columns=['title']).value_counts()
    #make dataframe look nice with proper index
    return  pd.DataFrame(venues_df, columns=['total'])

### Transport for London

In [2]:
transport_key = os.environ['TFL_API_KEY']

KeyError: 'TFL_API_KEY'

In [1]:
#This is a function that will intake string of API info we're looking for
#and a dictionary of the parameters to define
def TFL_API(search_term, parameters = {'app_key':transport_key}):
    """search_term must be a string of valid end point queries 
            - as specified by the TFL documentation
            
            returns JSON of results
            """
    
    root_endpoint = 'https://api.tfl.gov.uk/'
    
    #define endpoint
    endpoint = root_endpoint + search_term
    
    #GET
    response = re.get(root_endpoint + search_term, 
    params=parameters)
    
    #return status code and results
    status_code, results = response.status_code, response.json()
    
    #Let's make sure it worked
    if status_code != 200:
        print('Something went wrong!')
        print(status_code)
        
    return results

NameError: name 'transport_primary' is not defined